In [1]:
# Uncomment line below to install exlib
# !pip install exlib

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import tqdm
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch.nn as nn
import sentence_transformers

import sys
sys.path.append('/shared_data0/helenjin/exlib/src')
import exlib
from exlib.utils.emotion_helper import project_points_onto_axes, load_emotions
from exlib.datasets.emotion import load_data, load_model, EmotionDataset, EmotionClassifier, EmotionFixScore, get_emotion_scores

from exlib.features.text import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load datasets and pre-trained models

In [2]:
dataset = EmotionDataset("test")
model = EmotionClassifier()

mpressi/all_xlm-True


Some weights of XLMRobertaModel were not initialized from the model checkpoint at mpressi/all_xlm-True and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Model prediction

In [3]:
model.to(device)
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

In [4]:
model.eval()
for batch in tqdm(dataloader): 
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    utterances = [dataset.tokenizer.decode(input_id, skip_special_tokens=True) for input_id in input_ids]
    for utterance, label in zip(utterances, output.logits):
        print("Text: {}\nEmotion: {}\n".format(utterance, label.argmax()))
    break

  0%|          | 0/1357 [00:00<?, ?it/s]

Text: I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!
Emotion: 19

Text: It's wonderful because it's awful. At not with.
Emotion: 19

Text: Kings fan here, good luck to you guys! Will be an interesting game to watch! 
Emotion: 19

Text: I didn't know that, thank you for teaching me something today!
Emotion: 3



### Baselines
- Identity
- Random
- Words
- Phrases
- Sentences
- Clustering
- Archipelago

In [6]:
scores = get_emotion_scores(['word', 'phrase', 'sentence'])

100%|██████████| 1357/1357 [01:08<00:00, 19.80it/s]


In [7]:
for name in scores:
    metric = torch.tensor(scores[name])
    mean_metric = metric.nanmean()
    print(f'BASELINE {name} mean score: {mean_metric}')

BASELINE word mean score: 0.1346388711256625
BASELINE phrase mean score: 0.02088854563441117
BASELINE sentence mean score: 0.01617831451036725
